##  Note : <mark> checking for 20th june

### Installing libraries

In [145]:
import numpy as np
import pandas as pd
import datetime as dt
import os
pd.set_option('display.max_columns', None)

## reading files from folder

In [146]:
filesList = os.listdir("C:\\keshav\\office_report_daily\\dailyFiles")
filesList

['20june2023.csv', '21june2023.csv', '4May2023.csv']

In [147]:
clinetAmounts = pd.DataFrame()       #Excel 1
strategiesList = pd.DataFrame()       #Excel 2

for files in filesList:
    print(files)

    #calculating date
    s = files.split('.')[0]
    t = pd.to_datetime(s).date()
    date = t.strftime('%d-%m-%Y')
    print(date)


    #reading file and setting date as index
    df = pd.read_csv(f'C:\\keshav\\office_report_daily\\dailyFiles\\{files}')
    df['date'] = date
    df.set_index('date',inplace=True)

    #adding symbol column with help of exchange symbol
    df.insert(5,'Symbol',df['Exchange Symbol'].str.split().str.get(0))


    # filtering only for NIFTY,BANKNIFTY,FINNIFTY related to underlaying our dictionary
    df = df[df['Symbol'].isin(['NIFTY','BANKNIFTY','FINNIFTY'])]


    #calculating lot size column with help of classes 
    from packages.underlaying import lot_size
    df.insert(11,'lot_size',df['Symbol'].apply(lot_size))
    df.insert(13,'no_of_lots',(df['Qty']/df['lot_size']).astype(int))


    #calculating brokerage with help of class
    from packages.brokerage import brokerage
    df.insert(14,'brokerage_per_lot',df['Symbol'].apply(brokerage))
    df.insert(15,'brokerage_cost',df['no_of_lots']*df['brokerage_per_lot'])

    ## importing additional cost
    from packages.additionalCost import additional_cost
    df.insert(16,'additional_cost',(df['Symbol'].apply(additional_cost)*df['brokerage_cost']).astype(int))
    df.insert(17,'Total_cost',(df['brokerage_cost']+df['additional_cost']))
    pd.set_option('Display.max_columns',None)


    #calculating Txn
    def new_price(row):
        if row['Txn'] =='SELL':
            return (row['Avg Price']*row['Qty']-row['Total_cost'])/row['Qty']
        
        else:
            return (row['Avg Price']*row['Qty']+row['Total_cost'])/row['Qty']
    df['New Price']= df.apply(new_price,axis=1)



    def new_amount(row):
        if row['Txn'] == 'SELL':
            return row['New Price']*row['Qty']
        
        else:
            return -(row['New Price']*row['Qty'])

    df['New Amount']=df.apply(new_amount,axis=1)



    df.insert(2,'P_names',df['Portfolio Name'].str.extract('(CSL_STRANGS|70PCT|CP_2RE|25PCT_2RE)'))
    grp = df.groupby(['User Alias','P_names'])[['New Price','New Amount']].sum().reset_index()

    grp['date'] =date
    grp.set_index('date',inplace=True)

    #clients Total Amount as per date  ****************** 
    clientAmountsPivot = grp.pivot_table(index='date', columns='User Alias', values='New Amount', aggfunc='sum')
    clinetAmounts = pd.concat([clinetAmounts,clientAmountsPivot])


    strategies = grp.pivot_table(index='P_names', columns='User Alias', values='New Amount', aggfunc='sum').reset_index()
    strategies['date'] =date
    strategies.set_index('date',inplace=True)
    strategiesList = pd.concat([strategiesList,strategies])


    # break



20june2023.csv
20-06-2023
21june2023.csv
21-06-2023
4May2023.csv
04-05-2023


In [148]:
clinetAmounts = clinetAmounts.sort_index()

In [149]:
clinetAmounts

User Alias,AD1_IIFL,NMEDEL,RKEDEL,SS3POSIIFL,SIMULATED1
date,,,,,
04-05-2023,18795.0,NaN,18918.0,NaN,19376.25
20-06-2023,-12582.5,-22587.0,-22094.5,-21772.75,NaN
21-06-2023,-19287.0,-40070.5,-39494.0,-38768.00,NaN


In [150]:
clinetAmounts.to_excel('StoxxoDailyReport.xlsx')

In [151]:
# for dates as datetime indexes
# clinetAmounts = clinetAmounts.reset_index()
# clinetAmounts['date'] = pd.to_datetime(clinetAmounts['date'])
# clinetAmounts.set_index('date',inplace=True)
# clinetAmounts = clinetAmounts.sort_index()

##  to_excel

In [152]:
strategiesList

User Alias,P_names,AD1_IIFL,NMEDEL,RKEDEL,SS3POSIIFL,SIMULATED1
date,,,,,,
20-06-2023,25PCT_2RE,-4486.50,-5486.0,-5540.00,-5409.00,NaN
20-06-2023,70PCT,-4185.25,-8529.0,-8245.50,-8383.00,NaN
20-06-2023,CP_2RE,-4016.00,-8005.0,-8247.50,-7424.00,NaN
20-06-2023,CSL_STRANGS,105.25,-567.0,-61.50,-556.75,NaN
21-06-2023,25PCT_2RE,-6942.75,-14199.0,-13849.00,-13741.00,NaN
21-06-2023,70PCT,4412.25,8673.0,8677.50,8818.00,NaN
21-06-2023,CP_2RE,-9252.00,-19226.5,-19091.50,-18810.50,NaN
21-06-2023,CSL_STRANGS,-7504.50,-15318.0,-15231.00,-15034.50,NaN
04-05-2023,25PCT_2RE,8072.25,NaN,8277.00,NaN,8556.00


In [153]:
stg1 = strategiesList[strategiesList['P_names'] == '25PCT_2RE']
stg2 = strategiesList[strategiesList['P_names'] == '70PCT']
stg3 = strategiesList[strategiesList['P_names'] == 'CP_2RE']
stg4 = strategiesList[strategiesList['P_names'] == 'CSL_STRANGS']

In [154]:
stg1 = stg1.sort_index()
stg2 = stg2.sort_index()
stg3 = stg3.sort_index()
stg4= stg4.sort_index()

In [155]:
# Load the existing Excel file
existing_file = 'StoxxoDailyReport.xlsx'
book = pd.ExcelWriter(existing_file)

# Write the existing data to the Excel file
clinetAmounts.to_excel(book, sheet_name='clientReport')

# Add stg1 as a new sheet to the Excel file
stg1.to_excel(book, sheet_name='25PCT_2RE')
stg2.to_excel(book, sheet_name='70PCT')
stg3.to_excel(book, sheet_name='CP_2RE')
stg4.to_excel(book, sheet_name='CSL_STRANGS')


# Save and close the Excel file
book.close()
